In [9]:
import pandas as pd

In [10]:
teams = pd.read_csv('Fantrax-Team-Roster-League Against Humanity.csv')

In [11]:
teams['points'] = 0

In [12]:
teams.to_json('src/app/scieries.json', orient='records')